In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install -q transformers
%pip install -q loguru
%pip install -q openai langchain langchain[docarray] tiktoken unstructured
%pip install -q llama-index==0.10.4 chromadb loguru pypdf  sentence_transformers llama-index-embeddings-huggingface
%pip install -q pdf2image  pdfminer.six
%pip install -q pillow-heif
%pip install -q unstructured-inference
%pip install -q pikepdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
!wget -O Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
FILE_NAME = "./Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

--2024-02-14 19:24:01--  https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.81.15, 2620:100:6016:15::a27d:10f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876 [following]
--2024-02-14 19:24:01--  https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.81.15, 2620:100:6035:15::a27d:550f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.81.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221928 (217K) [appl

## Similarity K

In [4]:
K = 5

## LangChain

In [5]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from typing import Any, List
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML

In [6]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
#@title load data { vertical-output: true}
loader = UnstructuredPDFLoader(FILE_NAME)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
#@title splitting { vertical-output: true}
text_splitter = SentenceTransformersTokenTextSplitter()
texts = text_splitter.split_documents(docs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#@title indexing { vertical-output: true}
emb = OpenAIEmbeddings(openai_api_key=openai.api_key)
vec_db = Chroma.from_documents(documents=texts, embedding=emb)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
#@title LLM { vertical-output: true}
lc_model = ChatOpenAI(openai_api_key=openai.api_key, temperature=1.5)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [24]:
#@title base-retriever { vertical-output: true}
base_retriever = vec_db.as_retriever(search_kwargs={"k":K})

In [25]:
#@title MultiQueryRetriever { vertical-output: true}
final_retriever = MultiQueryRetriever.from_llm(base_retriever, lc_model)

In [26]:
#@title QA chain { vertical-output: true}
tmpl = """
You are an assistant to answer a question from user with a context.

Context:
{context}

Question:
{question}

The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
"""
prompt = ChatPromptTemplate.from_template(tmpl)
chain = {"question": RunnablePassthrough(), "context": final_retriever} \
        | prompt \
        | lc_model \
        | StrOutputParser() \

result = chain.invoke("Waht is the doc talking about?")

display(HTML(result))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the main topic of discussion in the document?', "2. Can you provide a brief description of the document's content?", '3. What is the subject matter presented in the document discussion?']


## Llama-Index

In [31]:
TEMPERATURE = 0.0
SIM_TOP_K = 3
RERANK_TOP_K = 3
WIN_SZ = 1024

In [32]:
import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.node_parser import SentenceWindowNodeParser

In [33]:
def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )

def create_vectors(path: str, collection_name="tmp_collection") -> Collection:
    chroma_client = chromadb.PersistentClient(path)
    # https://github.com/run-llama/llama_index/issues/6528
    return chroma_client.get_or_create_collection(collection_name)

def create_storage_context() -> StorageContext:
    path: str = "./db/LlamaIndexMultiVectorSummaryAgent"
    return StorageContext.from_defaults(
        vector_store=ChromaVectorStore(create_vectors(path=path))
    )


In [34]:
#@title load data { vertical-output: true}
input_files: List[str] = [FILE_NAME]
docs: SimpleDirectoryReader = SimpleDirectoryReader( input_files=input_files).load_data()

In [35]:
#@title LLM { vertical-output: true}
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [36]:
#@title splitting and indexing { vertical-output: true}
embs = "local:BAAI/bge-small-en-v1.5"

service_context: ServiceContext = (
    create_service_context(li_model, embs)
)

storage_context: StorageContext = (
    create_storage_context()
)

vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    #storage_context=storage_context,
    show_progress=True,
)

<ipython-input-33-0e0cc94f5ebb>:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  return ServiceContext.from_defaults(


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/160 [00:00<?, ?it/s]

In [37]:
#@title base-retriever { vertical-output: true}
base_retriever = vector_index.as_retriever(similarity_top_k=K)

In [60]:
#@title MultiQueriesRetriever { vertical-output: true}
class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3-turbo", temperature=1.5)
        prompt = self.template.format(question=query)
        res = self.model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        from typing import Dict
        query = query_bundle.query_str
        generated_queries = self.gen_queries(query)
        node_with_scores = await self.run_gen_queries(generated_queries)
        node_with_scores_uniqued = dict()
        # Simplely removing duplicated nodes in this notebook.
        # For Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        return node_with_scores_uniqued.values()



mr = MultiQueriesRetriever(base_retriever, li_model)

In [61]:
#@title { vertical-output: true}
ls = mr.gen_queries("Who is Bill Gates?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)
pretty_print("rls", rls)

ls


[
│   '1. Can you provide information about Bill Gates and his background?',
│   '2. What is known about the life and career of Microsoft co-founder Bill Gates?',
│   '3. Who is William Henry Gates III and what are his major accomplishments?',
│   '4. Could you tell me about the philanthropic activities and tech contributions of Bill Gates?',
│   '5. What are the notable achievements and personal history of the entrepreneur Bill Gates?'
]

100%|██████████| 5/5 [00:00<00:00, 16.91it/s]


rls


[
│   NodeWithScore(
│   │   node=TextNode(
│   │   │   id_='fbaa1662-e680-47cc-9d40-69f3a99c0eae',
│   │   │   embedding=None,
│   │   │   metadata={
│   │   │   │   'window': 'U.S.  securities laws in the same manner as if it were a U.S.  domestic \nreporting company.  Regulation FD provides that, when a company \ndiscloses material, non-public information about the company to securities \nmarket professionals or shareholders (where it is reasonably foreseeable \nthat the shareholders will trade on the information), that company must also \ninstantaneously disclose the information to the public.  "Securities market \nprofessionals" generally include analysts, institutional investors and other \ninvestment advisors.  \n \n Only the Investor Relations Manager is authorized to disclose information \nabout the Company in response to requests from securities market \nprofessionals or shareholders.  No Employee shall, without first obtaining \nexplicit approval from the Investor Relations Manager, communicate directly \nwith the investment community (including but not li mited to shareholders, \nanalysts, fund managers or potential investors) any information regarding \nthe Company, regardless whether such information has been disclosed \npublicly or not.  If an Employee receives a request for information from any \nsecurities mark et professionals or shareholders, he or she should promptly \ncontact the Investor Relations Manager to coordinate a response to such \nrequest.  \n o Communications with Media  \nNo Employee shall communicate with any media, publicly or privately, \nwithout first consul ting with the Company\'s personnel responsible for \nreleasing information.  In the event any issue arises that may emerge as a \npotential public relations emergency, such as customer\'s complaints written \non a public forum, Employees shall immediately report such incident to the \npersonnel responsible for releasing information.  If any member of the media \nasks for information about the Company or makes any other requests, \nEmployees shall refer their inquiries or requests to the Company\'s Investor \nRelations Manager .  For more detailed provisions regarding relationships \nwith the media, Employees shall refer to Rules for External \nCommunications by Senior Management and Other Employees to ensure \nthat every Employee will act in accordance with the prescribed conduct \nrules. \n 8.  Business Partners, Suppliers and Clients  \n\uf0a7 8.1 Procurement of Goods and Services We guarantee fair dealings with our \nsuppliers and we choose our suppliers through a fair evaluation of competitive bids. \n No Employee shall discriminate against or deceive a s upplier.  The decision to \nchoose a particular supplier shall be made by reference to the price, service, quality \nand reputation of the supplier as considered in the context of the Company\'s long-\nterm commercial interests.   \n ',
│   │   │   │   'original_text': '"Securities market \nprofessionals" generally include analysts, institutional investors and other \ninvestment advisors.  \n \n',
│   │   │   │   'page_label': '6',
│   │   │   │   'file_name': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_path': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_type': 'application/pdf',
│   │   │   │   'file_size': 221928,
│   │   │   │   'creation_date': '2024-02-14',
│   │   │   │   'last_modified_date': '2024-02-14',
│   │   │   │   'last_accessed_date': '2024-02-14'
│   │   │   },
│   │   │   excluded_embed_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   │   'creation_date',
│   │   │   │   'last_modified_date',
│   │   │   │   'last_accessed_date',
│   │   │   │   'window',
│   │   │   │   'original_text'
│   │   │   ],
│   │   │   excluded_llm_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   

In [62]:
#@title QA query { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
final_res = await RetrieverQueryEngine(mr).aquery(query_text)
display(HTML(final_res.response))

100%|██████████| 5/5 [00:00<00:00, 18.96it/s]
